In [1]:
from DeepPurpose import utils, dataset, CompoundPred
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings("ignore")

X_drugs, y, drugs_index = dataset.load_HIV(path = './data')
drug_encoding = 'Morgan'

import numpy as np 
import os
import sys
import pandas as pd 
from ConfSelect import weighted_BH, weighted_CS, eval_sel 

ModuleNotFoundError: No module named 'DeepPurpose'

In [48]:
seed = 3
q = 0.5

np.random.seed(seed)
n = len(y)
reind = np.random.permutation(n)

X_drugs_train = X_drugs[reind[0:int(n*0.4+1)]]
y_train = y[reind[0:int(n*0.4+1)]]
X_drugs_other = X_drugs[reind[int(1+n*0.4):n]]
y_other = y[reind[int(1+n*0.4):n]]

In [49]:
# =============================================================================
# # train prediction model on the training data
# =============================================================================

ttrain, tval, ttest = utils.data_process(X_drug = X_drugs_train, y = y_train, 
                                         drug_encoding = drug_encoding,
                                         split_method='random',frac=[1-1e-8, 1e-8, 1e-8],
                                         random_seed = seed)
 
# small neural network
#config = utils.generate_config(drug_encoding = drug_encoding, 
#                       cls_hidden_dims = [1024,1024,512], 
#                       train_epoch = 3, 
#                       LR = 0.001, 
#                       batch_size = 128,
#                       hidden_dim_drug = 128,
#                       mpnn_hidden_size = 128,
#                       mpnn_depth = 3
#                      )
#model = CompoundPred.model_initialize(**config)
#model.train(ttrain, tval, ttest)
# Example transformation assuming 'drug_encoding' contains arrays of the same length
if isinstance(ttrain['drug_encoding'].iloc[0], (list, np.ndarray)):
    # Convert each array/list to a row in a new DataFrame
    drug_features = pd.DataFrame(ttrain['drug_encoding'].tolist())
    # You can now concatenate these features back to your original DataFrame (if needed) or use as is for modeling
    ttrain_processed = pd.concat([ttrain.drop(columns=['drug_encoding']), drug_features], axis=1)
ttrain_processed = ttrain_processed.apply(pd.to_numeric, errors='coerce')
# Assuming ttrain_processed is your DataFrame after processing
ttrain_processed.columns = ttrain_processed.columns.astype(str)

# Drop the 'SMILES' column as it's not required for modeling
ttrain_processed = ttrain_processed.drop(columns=['SMILES'])

# Separate the features and the label
t_train = ttrain_processed.drop(columns=['Label'])

# Initialize Random Forest assifier
model = RandomForestClassifier(n_estimators=100, random_state=seed)

# Train the Random Forest model
model.fit(t_train, y_train)
#print(ttrain.drug_encoding)

Drug Property Prediction Mode...
in total: 16451 drugs
encoding drug...
unique drugs: 16451


[23:39:25] WARNING: not removing hydrogen atom without neighbors
[23:39:25] WARNING: not removing hydrogen atom without neighbors


Done.


RandomForestClassifier(random_state=3)

In [50]:
print(t_train)

         0    1    2    3    4    5    6    7    8    9  ...  1014  1015  \
0      0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0   
1      0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0   
2      0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0   
3      0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0   
4      1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0   
...    ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   ...   ...   
16446  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0   
16447  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0   
16448  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0   
16449  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0   
16450  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0   

       1016  1017  1018  1019  1020  1021  1022  1023  
0       0.0   1.0   1.0   1.0  

In [51]:
# =============================================================================
# # weighted split into calibration and test data
# =============================================================================

d_, d__, dother = utils.data_process(X_drug = X_drugs_other, y = y_other, 
                                     drug_encoding = drug_encoding,
                                     split_method='random',frac=[1e-8, 1e-8, 1-2e-8],
                                     random_seed = seed)

# Example transformation assuming 'drug_encoding' contains arrays of the same length
if isinstance(dother['drug_encoding'].iloc[0], (list, np.ndarray)):
    # Convert each array/list to a row in a new DataFrame
    drug_features = pd.DataFrame(dother['drug_encoding'].tolist())
    # You can now concatenate these features back to your original DataFrame (if needed) or use as is for modeling
    dother_processed = pd.concat([dother.drop(columns=['drug_encoding']), drug_features], axis=1)
dother_processed = dother_processed.apply(pd.to_numeric, errors='coerce')
# Assuming ttrain_processed is your DataFrame after processing
dother_processed.columns = dother_processed.columns.astype(str)

# Drop the 'SMILES' column as it's not required for modeling
dother_processed = dother_processed.drop(columns=['SMILES'])

# Separate the features and the label
d_other = dother_processed.drop(columns=['Label'])

all_pred = model.predict(d_other)
train_pred = model.predict(t_train)

p_x = np.minimum(0.8, np.exp(all_pred - np.mean(train_pred)) / (1+np.exp(all_pred - np.mean(train_pred))))
in_calib = np.random.binomial(1, p_x, size=len(p_x))

dcalib = d_other[pd.Series(in_calib==1).values]
dtest = d_other[pd.Series(in_calib==0).values]

dcalib = dcalib.reset_index(drop=True)
dtest = dtest.reset_index(drop=True)

hat_mu_calib = np.array(model.predict(dcalib))
hat_mu_test = np.array(model.predict(dtest))

dcalib2 = dother[pd.Series(in_calib==1).values]
dtest2 = dother[pd.Series(in_calib==0).values]

y_calib = np.array(dcalib2["Label"])
w_calib = np.array(1/p_x[in_calib==1] - 1)
y_test = np.array(dtest2['Label'])
w_test = np.array(1/p_x[in_calib==0] - 1)

Drug Property Prediction Mode...
in total: 24676 drugs
encoding drug...
unique drugs: 24676
Done.


In [52]:
# Calculate MSE
mse = np.mean((y_test - hat_mu_test) ** 2)
print(f"Mean Squared Error: {mse}")

# Calculate R^2 Score
y_mean = np.mean(y_test)
ss_tot = np.sum((y_test - y_mean) ** 2)
ss_res = np.sum((y_test - hat_mu_test) ** 2)
r2 = 1 - (ss_res / ss_tot)
print(f"R^2 Score: {r2}")

Mean Squared Error: 0.02707899375100144
R^2 Score: 0.07074893312408814


In [53]:
# =============================================================================
# # run testing procedures
# =============================================================================

c = 0.0
 
calib_scores_res = y_calib - hat_mu_calib
calib_scores_sub = - hat_mu_calib 
calib_scores_clip = 100 * (y_calib > c) + c * (y_calib <= c) - hat_mu_calib
 
test_scores = c - hat_mu_test

 
# ========================= 
# ## weighted BH procedure
# ========================= 

# use scores res, sub, and clip
BH_res = weighted_BH(calib_scores_res, w_calib, test_scores, w_test, q)  
BH_sub = weighted_BH(calib_scores_sub[y_calib <= c], w_calib[y_calib<=c], test_scores, w_test, q) 
BH_clip = weighted_BH(calib_scores_clip, w_calib, test_scores, w_test, q)



In [56]:
# ==================================== 
# ## weighted conformalized selection
# ==================================== 

#hete
# Adjusted code to match the actual function output
#WCS_res_0_hete, WCS_res_hete, WCS_pvals_hete = weighted_CS(calib_scores_res, w_calib, test_scores, w_test,q, rand='hete')
# Assuming 'rand' is defined earlier or you choose a method ('hete', 'homo', 'dtm')
#WCS_sub_0_hete, WCS_sub_hete, WCS_sub_pvals_hete = weighted_CS(calib_scores_sub[y_calib <= c], 
#                                                        w_calib[y_calib<=c], 
#                                                        test_scores, w_test, q, rand='hete')
#WCS_clip_0_hete, WCS_clip_hete, WCS_clip_pvals_hete = weighted_CS(calib_scores_clip, 
#                                                           w_calib, test_scores, w_test, q, rand='hete')

##homo
#WCS_res_0_homo, WCS_res_homo, WCS_pvals_homo = weighted_CS(calib_scores_res, w_calib, test_scores, w_test,q, rand='homo')
# Assuming 'rand' is defined earlier or you choose a method ('homo', 'homo', 'dtm')
#WCS_sub_0_homo, WCS_sub_homo, WCS_sub_pvals_homo = weighted_CS(calib_scores_sub[y_calib <= c], 
 #                                                       w_calib[y_calib<=c], 
  #                                                      test_scores, w_test, q, rand='homo')
#WCS_clip_0_homo, WCS_clip_homo, WCS_clip_pvals_homo = weighted_CS(calib_scores_clip, 
 #                                                          w_calib, test_scores, w_test, q, rand='homo')

#dtm
WCS_res_0_dtm, WCS_res_dtm, WCS_pvals_dtm = weighted_CS(calib_scores_res, w_calib, test_scores, w_test,q, rand='dtm')
# Assuming 'rand' is defined earlier or you choose a method ('hete', 'homo', 'dtm')
WCS_sub_0_dtm, WCS_sub_dtm, WCS_sub_pvals_dtm = weighted_CS(calib_scores_sub[y_calib <= c], 
                                                        w_calib[y_calib<=c], 
                                                        test_scores, w_test, q, rand='dtm')
WCS_clip_0_dtm, WCS_clip_dtm, WCS_clip_pvals_dtm = weighted_CS(calib_scores_clip, 
                                                           w_calib, test_scores, w_test, q, rand='dtm')
# use scores res, sub, and clip
#WCS_res_0, WCS_res_hete, WCS_res_homo, WCS_res_dtm = weighted_CS(calib_scores_res, 
                                                                 #w_calib, test_scores, w_test, q)
#WCS_sub_0, WCS_sub_hete, WCS_sub_homo, WCS_sub_dtm = weighted_CS(calib_scores_sub[y_calib <= c],
                                                                 #w_calib[y_calib<=c], 
                                                                 #test_scores, w_test, q)
#WCS_clip_0, WCS_clip_hete, WCS_clip_homo, WCS_clip_dtm = weighted_CS(calib_scores_clip, 
                                                                     #w_calib, test_scores,w_test, q)

In [57]:
# =============================================================================
# # summarize FDP, power and selection sizes
# =============================================================================


#BH_res_fdp, BH_res_power = eval_sel(BH_res, y_test, np.array([c]*len(y_test)))
#BH_sub_fdp, BH_sub_power = eval_sel(BH_sub, y_test, np.array([c]*len(y_test)))
#BH_clip_fdp, BH_clip_power = eval_sel(BH_clip, y_test, np.array([c]*len(y_test))) 

# Assuming BH_res[0] contains the integer indices you want to use for evaluation
BH_res_indices = BH_res[0]
# Now pass these indices to the eval_sel function
BH_res_fdp, BH_res_power = eval_sel(BH_res_indices, y_test, np.array([c]*len(y_test)))

# Assuming BH_sub[0] contains the integer indices for the "sub" case
BH_sub_indices = BH_sub[0]
# Now pass these indices to the eval_sel function
BH_sub_fdp, BH_sub_power = eval_sel(BH_sub_indices, y_test, np.array([c]*len(y_test)))

# Assuming BH_clip[0] contains the integer indices for the "clip" case
BH_clip_indices = BH_clip[0]
# Now pass these indices to the eval_sel function
BH_clip_fdp, BH_clip_power = eval_sel(BH_clip_indices, y_test, np.array([c]*len(y_test)))



all_BH = [BH_res_indices, BH_sub_indices, BH_clip_indices]
all_sel = [[WCS_res_hete, WCS_res_homo, WCS_res_dtm], 
           [WCS_sub_hete, WCS_sub_homo, WCS_sub_dtm],
           [WCS_clip_hete, WCS_clip_homo, WCS_clip_dtm]]
fdp = [BH_res_fdp, BH_sub_fdp, BH_clip_fdp]
power = [BH_res_power, BH_sub_power, BH_clip_power] 
ndiff = [0] * 3
nsel = [len(BH_res_indices), len(BH_sub_indices), len(BH_clip_indices)]
nsame = [len(BH_res_indices), len(BH_sub_indices), len(BH_clip_indices)]

for ii in range(3):
    sels = all_sel[ii]
    tpowers = []
    tfdps = []
    tnsels = []
    tndiffs = []
    tnsames = []
    for jj in range(3):
        tfdp, tpower = eval_sel(sels[jj], y_test, np.array([c]*len(y_test)))
        tpowers.append(tpower)
        tfdps.append(tfdp)
        tnsels.append(len(sels[jj]))
        tndiffs.append(len(np.setxor1d(all_BH[ii], sels[jj])))
        tnsames.append(len(np.intersect1d(all_BH[ii], sels[jj])))
    fdp += tfdps
    power += tpowers
    ndiff += tndiffs
    nsel += tnsels
    nsame += tnsames
 

res = pd.DataFrame({"FDP": fdp, "power": power, "nsel": nsel, "ndiff": ndiff, "nsame": nsame,
                       "score": ["res", "sub", "clip"] + ["res"]*3 + ["sub"]*3 + ["clip"]*3,
                       "method": ["WBH"]*3 + ['WCS.hete', 'WCS.homo', "WCS.dtm"] *3,
                                   "q": q, "seed": seed})

In [58]:
save_path = "./DPP_results"
if not os.path.exists(save_path): 
   os.makedirs(save_path)

res.to_csv("./DPP_results/seedrf"+str(seed)+"q"+str(q)+".csv")

In [67]:
print(pd.Series(in_calib==1).values[0:10])

[ True  True False  True  True  True  True False False False]


In [32]:
print(ttrain_processed)

       SMILES  Label    0    1    2    3    4    5    6    7  ...  1014  1015  \
0         NaN      0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0   
1         NaN      0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0   
2         NaN      0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0   
3         NaN      0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0   
4         NaN      0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0   
...       ...    ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   ...   ...   
16446     NaN      0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0   
16447     NaN      0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0   
16448     NaN      0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0   
16449     NaN      0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0   
16450     NaN      0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0   

       1016  1017  1018  10

In [104]:
print(dcalib.shape)
print(dtest.shape)
print(dother.shape)

(12356, 3)
(12320, 3)
(24676, 3)


In [105]:
# It might be useful to check the distribution of in_calib
print(np.mean(in_calib))  # This prints the proportion of data marked for calibration

0.5007294537202139


In [78]:
print(dother.columns)
print(dcalib.columns)
print(dtest.columns)

Index(['SMILES', 'Label', 'drug_encoding'], dtype='object')
Index(['SMILES', 'Label', 'drug_encoding'], dtype='object')
Index(['SMILES', 'Label', 'drug_encoding'], dtype='object')


In [76]:
hat_mu_calib = np.array(model.predict(dother.iloc[0:100]))
#hat_mu_test = np.array(model.predict(dtest.iloc[:10]))

predicting...


In [39]:
print(dcalib.head())
print(dtest.head())

                                               SMILES  Label  \
0   [Br-].c1ccc([P+](CCCCCC[P+](c2ccccc2)(c2ccccc2...      0   
1   Clc1ccc(C=Nc2ccc(SSc3ccc(N=Cc4ccc(Cl)cc4)cc3)c...      0   
6   Oc1c(OCc2nnc(CCCCCCCCc3nnc(COc4ccc5ccccc5c4O)o...      0   
10                   Cc1cn(Cc2ccccc2CCO)c(=S)[nH]c1=O      0   
12  Cc1cn(C2CC(N=[N+]=[N-])C(COC(=O)CCCCCCCCCCOc3c...      1   

                                        drug_encoding  
0   [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...  
1   [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...  
6   [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...  
10  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...  
12  [0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...  
                                              SMILES  Label  \
2                              CCOC(=O)C=CSCCC(=O)OC      0   
3          CCn1cc(C(=O)O)c(=O)c2cc(F)c(-n3cccc3)cc21      0   
4                     [C-]#[N+]CS(=O)(=O)c1ccc(C)cc1      0   
5  Cc1cc(-c2ccc(C(=O)C(c3cc

In [19]:
# Check if 'drug_encoding' column exists
print('drug_encoding' in dcalib.columns)
print('drug_encoding' in dtest.columns)

True
True


In [20]:
# Predict with a single sample to test
single_sample = dcalib.iloc[0:1]  # Adjust based on the expected input format
model.predict(single_sample)

predicting...


[0.02133091911673546]

In [54]:
model.predict(dcalib.iloc[1:2])

predicting...


IndexError: single positional indexer is out-of-bounds

In [46]:
print(dcalib.iloc[2])

SMILES           Oc1c(OCc2nnc(CCCCCCCCc3nnc(COc4ccc5ccccc5c4O)o...
Label                                                            0
drug_encoding    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
Name: 6, dtype: object


In [29]:
print(dcalib.iloc[1])

SMILES           Clc1ccc(C=Nc2ccc(SSc3ccc(N=Cc4ccc(Cl)cc4)cc3)c...
Label                                                            0
drug_encoding    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
Name: 1, dtype: object
